In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import sys
import os
import random
import shutil

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, adjusted_rand_score
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical, set_random_seed
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from pyspark.sql.functions import col, udf, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession

2024-09-24 19:51:36.607117: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-24 19:51:36.610537: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-24 19:51:36.622341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-24 19:51:36.641789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-24 19:51:36.649498: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-24 19:51:36.665912: I tensorflow/core/platform/cpu_feature_gu

## En local sur mon ordinateur, sans PySpark

In [2]:
base_model = MobileNetV2(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3))

In [3]:
def create_model(base_model, nbr_de_classes):
    for layer in base_model.layers:
        layer.trainable = False
    # Ajout des couches de classification
    model = Sequential([base_model,
                        GlobalAveragePooling2D(),
                        Dense(1024, activation='relu'),
                        Dense(nbr_de_classes, activation='softmax')
                       ])
    return model

In [4]:
model = create_model(base_model, 3)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         3,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,572,803 (13.63 MB)

 Trainable params: 1,314,819 (5.02 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [3]:
def metrics_display(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='macro')
    recall = recall_score(true_labels, predicted_labels, average='macro')
    f1 = f1_score(true_labels, predicted_labels, average='macro')
    ARI = adjusted_rand_score(true_labels, predicted_labels)
    print(f'accuracy = {accuracy}   precision = {precision}   recall={recall}   f1 = {f1}   ARI = {ARI}')

In [39]:
def train_and_test_model(train_dir,
                         test_dir,
                         model,
                         seed_value=42,
                         target_size=(224, 224),
                         optimizer=Nadam,
                         learning_rate=0.001,
                         epochs=10
                        ):
    
    # Fixation des germes des générateurs aléatoires pour assurer la reproductibilité des résultats.
    set_random_seed(seed_value)
    
    # Préparation du générateur de données
    generator = ImageDataGenerator(rescale=1./255)
    
    train_generator = generator.flow_from_directory(train_dir,
                                                    target_size=target_size,
                                                    class_mode='categorical',
                                                    seed=seed_value
                                                    )

    test_generator = generator.flow_from_directory(test_dir,
                                                   target_size=target_size,
                                                   class_mode='categorical',
                                                   shuffle=False,
                                                   seed=seed_value
                                                   )
    
    # Compiler le modèle
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Construction des callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

    # Entraînement du modèle
    history = model.fit(train_generator,
                        validation_data=test_generator,
                        callbacks=[early_stopping, reduce_lr],
                        epochs=epochs)

    # Obtention des étiquettes prédites et réelles
    predictions = model.predict(test_generator)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_generator.classes
    
    # Évaluation des performances du modèle
    metrics_display(y_true, y_pred)

In [4]:
train_dir = '../data/train_local'
test_dir = '../data/test_local'

In [44]:
train_and_test_model(train_dir, test_dir, model)

Found 1381 images belonging to 3 classes.
Found 461 images belonging to 3 classes.
Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 44s 920ms/step - accuracy: 0.8996 - loss: 0.1968 - val_accuracy: 1.0000 - val_loss: 1.2202e-04 - learning_rate: 0.0010
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 40s 906ms/step - accuracy: 1.0000 - loss: 2.9620e-05 - val_accuracy: 1.0000 - val_loss: 6.4307e-05 - learning_rate: 0.0010
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 40s 902ms/step - accuracy: 1.0000 - loss: 6.8215e-06 - val_accuracy: 1.0000 - val_loss: 3.1388e-05 - learning_rate: 0.0010
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 44s 996ms/step - accuracy: 1.0000 - loss: 3.3063e-06 - val_accuracy: 1.0000 - val_loss: 2.3720e-05 - learning_rate: 0.0010
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 41s 925ms/step - accuracy: 1.0000 - loss: 2.1207e-06 - val_accuracy: 1.0000 - val_loss: 1.6217e-05 - learning_rate: 5.0000e-04
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 40s 900ms/step - accuracy: 1.0000 - loss: 1.4989e-06 - val_accuracy: 1.000

## En local sur mon ordinateur, avec PySpark

In [5]:
env_projet = '/home/chakir/.local/share/virtualenvs/Projet8-N0cGnpCf/bin/python'

In [6]:
# Configuration de l'environnement virtuel pour PySpark
os.environ['PYSPARK_PYTHON'] = env_projet
os.environ['PYSPARK_DRIVER_PYTHON'] = env_projet

In [7]:
# Démarrage d'une session Spark
spark = (SparkSession.builder
                     .appName('P8')
                     .master('local')
                     .config('spark.executorEnv.PYSPARK_PYTHON', env_projet)
                     .getOrCreate())

24/09/17 14:03:21 WARN Utils: Your hostname, chakir-Latitude-7390 resolves to a loopback address: 127.0.1.1; using 192.168.43.175 instead (on interface wlp2s0)
24/09/17 14:03:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/17 14:03:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/17 14:03:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/17 14:03:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [8]:
sc = spark.sparkContext

In [9]:
spark

In [11]:
images_train = (spark.read.format('binaryFile')
                     .option('pathGlobFilter', '*.jpg')
                     .option('recursiveFileLookup', 'true')
                     .load(train_dir))

images_test = (spark.read.format('binaryFile')
                    .option('pathGlobFilter', '*.jpg')
                    .option('recursiveFileLookup', 'true')
                    .load(test_dir))

In [12]:
images_train = images_train.withColumn('label', element_at(split(images_train['path'], '/'),-2))
images_test = images_test.withColumn('label', element_at(split(images_test['path'], '/'),-2))

In [13]:
print(images_train.printSchema())
print(images_test.select('path','label').show(5,False))

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-------------------------------------------------------------------------------------+--------------+
|path                                                                                 |label         |
+-------------------------------------------------------------------------------------+--------------+
|file:/home/chakir/Bureau/PROJETS/Projet8/data/test_local/Apple Braeburn/r_326_100.jpg|Apple Braeburn|
|file:/home/chakir/Bureau/PROJETS/Projet8/data/test_local/Apple Braeburn/r_4_100.jpg  |Apple Braeburn|
|file:/home/chakir/Bureau/PROJETS/Projet8/data/test_local/Apple Braeburn/r_8_100.jpg  |Apple Braeburn|
|file:/home/chakir/Bureau/PROJETS/Projet8/data/test_local/Apple Braeburn/r_324_100.jpg|Apple Braeburn|
|file:/home/chakir/Bureau/PROJETS/Projet8/data/test_local/Apple Braeb

In [14]:
def create_model_spark(spark_context, base_model, nbr_de_classes):
    for layer in base_model.layers:
        layer.trainable = False
    # Ajout des couches de classification
    model = Sequential([base_model,
                        GlobalAveragePooling2D(),
                        Dense(1024, activation='relu'),
                        Dense(nbr_de_classes, activation='softmax')
                       ])
    brodcast_weights = spark_context.broadcast(model.get_weights())
    return model

In [15]:
# Fonction de prétraitement d'images (redimensionnement, normalisation)
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array /= 255.0
    return img_array

# Fonction pour obtenir les chemins des images et leurs étiquettes
def get_image_paths_and_labels(spark_session, dir_path):
    
    images = (spark_session.read.format('binaryFile')
                           .option('pathGlobFilter', '*.jpg')
                           .option('recursiveFileLookup', 'true')
                           .load(dir_path))

    images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
    
    return images

In [16]:
def train_and_test_model_spark(train_dir, 
                               test_dir, 
                               model,
                               spark_session,
                               seed_value=42, 
                               target_size=(224, 224), 
                               optimizer=Nadam, 
                               learning_rate=0.001, 
                               epochs=10):
    
    # Fixation des germes pour la reproductibilité des résultats
    set_random_seed(seed_value)
    
    # Obtention des chemins des images et leurs étiquettes
    train_df = get_image_paths_and_labels(spark_session, train_dir)
    test_df = get_image_paths_and_labels(spark_session, test_dir)
    
    # Conversion en dataframe pandas pour prétraitement en local
    train_data = train_df.toPandas()
    test_data = test_df.toPandas()

    # Nettoyage des chemins de fichiers en retirant le préfixe "file:"
    train_data['path'] = train_data.path.apply(lambda x: x.replace('file:', ''))
    test_data['path'] = test_data.path.apply(lambda x: x.replace('file:', ''))
    
    # Prétraitement des images
    X_train = np.array([preprocess_image(path) for path in train_data.path])
    X_test = np.array([preprocess_image(path) for path in test_data.path])

    # Sélection des étiquettes d'entraînement et de test
    y_train = train_data.label
    y_test = test_data.label
    
    # Encodage des étiquettes en one-hot
    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train.values)
    y_test = encoder.transform(y_test.values)
    y_train_cnn = to_categorical(y_train)
    y_test_cnn = to_categorical(y_test)

    # Compiler le modèle
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Callbacks pour le modèle
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
    
    # Entraîner le modèle
    history = model.fit(X_train, y_train_cnn,
                        validation_data=(X_test, y_test_cnn),
                        callbacks=[early_stopping, reduce_lr],
                        epochs=epochs)
    
    # Prédictions
    predictions = model.predict(X_test)
    y_pred = np.argmax(predictions, axis=1)
    
    # Évaluation du modèle
    metrics_display(y_test, y_pred)

In [38]:
model = create_model_spark(sc, base_model, 3)

In [39]:
train_and_test_model_spark(train_dir, test_dir, model, spark)

Epoch 1/10


2024-09-17 14:41:02.977600: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 831516672 exceeds 10% of free system memory.


44/44 ━━━━━━━━━━━━━━━━━━━━ 47s 986ms/step - accuracy: 0.8898 - loss: 0.2945 - val_accuracy: 1.0000 - val_loss: 3.2307e-04 - learning_rate: 0.0010
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 41s 928ms/step - accuracy: 1.0000 - loss: 5.2905e-05 - val_accuracy: 1.0000 - val_loss: 1.7070e-04 - learning_rate: 0.0010
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 42s 957ms/step - accuracy: 1.0000 - loss: 1.9368e-05 - val_accuracy: 1.0000 - val_loss: 1.0505e-04 - learning_rate: 0.0010
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 43s 976ms/step - accuracy: 1.0000 - loss: 1.0073e-05 - val_accuracy: 1.0000 - val_loss: 7.2057e-05 - learning_rate: 0.0010
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 1.0000 - loss: 6.2570e-06 - val_accuracy: 1.0000 - val_loss: 4.5939e-05 - learning_rate: 0.0010
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 1.0000 - loss: 4.2515e-06 - val_accuracy: 1.0000 - val_loss: 3.8880e-05 - learning_rate: 0.0010
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 41s 936ms/step - 

## Sur le cloud AWS

In [8]:
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

In [4]:
env_projet = sys.executable

In [5]:
# Configuration de l'environnement virtuel pour PySpark
os.environ['PYSPARK_PYTHON'] = env_projet
os.environ['PYSPARK_DRIVER_PYTHON'] = env_projet

In [10]:
# Démarrage d'une session Spark
spark = (SparkSession.builder
                     .appName('P8')
                     .master('yarn')
                     .config('spark.executorEnv.PYSPARK_PYTHON', env_projet)
                     .config('spark.executorEnv.PYSPARK_DRIVER_PYTHON', env_projet)
                     .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/24 18:10:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/24 18:10:17 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [11]:
sc = spark.sparkContext

In [12]:
spark

In [31]:
train_dir = 's3a://chakir-p8-data/data/train_local'
test_dir = 's3a://chakir-p8-data/data/test_local'

In [27]:
def preprocess_spark(row, target_size=(224, 224)):
        image_path = row['path']
        img = load_img(image_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array /= 255.0
        return img_array, row['label']

In [28]:
def train_and_test_model_spark(train_dir, 
                               test_dir, 
                               model,
                               spark_session,
                               seed_value=42, 
                               target_size=(224, 224), 
                               optimizer=Nadam, 
                               learning_rate=0.001, 
                               epochs=10):
    
    # Fixation des germes pour la reproductibilité des résultats
    set_random_seed(seed_value)
    
    # Obtention des chemins des images et leurs étiquettes
    train_df = get_image_paths_and_labels(spark_session, train_dir)
    test_df = get_image_paths_and_labels(spark_session, test_dir)
    
    # Transformation des données en parallèle sur Spark
    train_data_rdd = train_df.rdd.map(preprocess_spark)
    test_data_rdd = test_df.rdd.map(preprocess_spark)

    # Collecte des données en mémoire locale pour l'entraînement du modèle
    train_data = train_data_rdd.collect()
    test_data = test_data_rdd.collect()
    
    # Séparation des données et des étiquettes
    X_train, y_train = zip(*train_data)
    X_test, y_test = zip(*test_data)

    X_train = np.array(X_train)
    X_test = np.array(X_test)
    
    # Encodage des étiquettes en one-hot
    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train.values)
    y_test = encoder.transform(y_test.values)
    y_train_cnn = to_categorical(y_train)
    y_test_cnn = to_categorical(y_test)

    # Compiler le modèle
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Callbacks pour le modèle
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
    
    # Entraîner le modèle
    history = model.fit(X_train, y_train_cnn,
                        validation_data=(X_test, y_test_cnn),
                        callbacks=[early_stopping, reduce_lr],
                        epochs=epochs)
    
    # Prédictions
    predictions = model.predict(X_test)
    y_pred = np.argmax(predictions, axis=1)
    
    # Évaluation du modèle
    metrics_display(y_test, y_pred)

In [29]:
model = create_model_spark(sc, base_model, 3)

In [32]:
train_and_test_model_spark(train_dir, test_dir, model, spark)

24/09/24 19:30:29 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://chakir-p8-data/data/train_local.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.reso

Py4JJavaError: An error occurred while calling o71.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Da

In [2]:
spark = SparkSession.builder \
    .appName('P8') \
    .getOrCreate()

# Tester l'accès à S3
df = spark.read.text("s3a://chakir-p8-data/requirements.txt")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/24 19:51:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/24 19:51:51 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://chakir-p8-data/requirements.txt.
java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org

Py4JJavaError: An error occurred while calling o26.text.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Da